In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import matplotlib.pyplot as pl
import examples.utils as utils

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = 'examples/data/birth_life_2010.txt'

In [3]:
data, n_samples = utils.read_birth_life_data(data)

In [4]:
np.shape(data)

(190, 2)

In [5]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, name = "X")
Y = tf.placeholder(tf.float32, name = "Y")

In [6]:
w = tf.get_variable("weights", initializer = tf.constant(0.0))
b = tf.get_variable("bias", initializer = tf.constant(0.0))
Y_pred = w*X+b
loss = tf.square(Y - Y_pred, name = "loss")

In [7]:
opt = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
optimizer = opt.minimize(loss)

In [8]:
opgraph = './graphs/linear_reg_'+ time.strftime('%Y%m%d_%H%M',(time.gmtime(time.time()-60*60*7)))
writer = tf.summary.FileWriter(opgraph, tf.get_default_graph())

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(100):
        total_loss = 0
        for x, y in data:
            _,l = sess.run([optimizer,loss], feed_dict = {X:x,Y:y})
            total_loss = total_loss + l
        
        if(i%20 == 0):
            print("Epoch {0}: Average Loss:{1} ".format(i,total_loss/n_samples))
    
    
    writer.close()
    w_out, b_out = sess.run([w,b])

Epoch 0: Average Loss:1661.8637834631543 
Epoch 20: Average Loss:119.20935661137888 
Epoch 40: Average Loss:37.305591759538146 
Epoch 60: Average Loss:30.524590178362192 
Epoch 80: Average Loss:30.04458791257593 


In [21]:
tf.reset_default_graph()
dataset = tf.data.Dataset.from_tensor_slices((data[:,0],data[:,1]))
iterator = dataset.make_initializable_iterator()
x,y = iterator.get_next()


w = tf.get_variable(name = "weight", initializer = tf.constant(0.0))
b = tf.get_variable(name = "bias", initializer = tf.constant(0.0))


ypred = w*x + b
loss = tf.square(y-ypred, name = "loss")
opt = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(100):
        sess.run(iterator.initializer)
        total_loss = 0
        k = 0
        try:
            while True:
                _,l = sess.run([opt,loss])
                total_loss += l
                k = k+1
        except tf.errors.OutOfRangeError:
            pass
        
        if(i%20 == 0):
            print("Epoch - {0}, Average loss = {1}".format(i,total_loss/n_samples))
            
    
    w_out, b_out = sess.run([w,b])
    

Epoch - 0, Average loss = 1661.8637834631543
Epoch - 20, Average loss = 119.20935661137888
Epoch - 40, Average loss = 37.305591759538146
Epoch - 60, Average loss = 30.524590178362192
Epoch - 80, Average loss = 30.04458791257593
